In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import (davies_bouldin_score, silhouette_score, calinski_harabasz_score)
pd.options.display.max_columns=150

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [5]:
data =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UNSW-NB15_1.csv', header = None)
datacols = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NUSW-NB15_features.csv',encoding='cp1252')
columns = datacols.Name
data.columns = columns

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/UNSW-NB15_1.csv'

In [5]:
data.head()

Name,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,29,0,0,dns,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.89101,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0,0,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0,0,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.00000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0,0,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.00000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0,0,0,7,9,1,1,1,1,1,NaN,0


In [6]:
data['service']= np.where(data['service'] == '-', 'None', data['service'])
data['state']= np.where(data['state'] == 'no', 'None', data['state'])
data['is_sm_ips_ports'] = data['is_sm_ips_ports'].astype(bool)
data['is_ftp_login'] = data['is_ftp_login'].astype(bool)
data['ct_state_ttl'] = data['ct_state_ttl'].astype('category')

In [18]:
num_cols=data.select_dtypes(include='number')
num_cols.columns

Index(['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'Sload',
       'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'ct_flw_http_mthd', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst',
       'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'Label'],
      dtype='object', name='Name')

In [ ]:
num_cols = num_cols.drop(columns='Label', axis=1)

In [21]:

scaler = MinMaxScaler()
num_data = scaler.fit_transform(num_cols)

In [22]:
num_data = pd.DataFrame(num_data)

In [23]:
num_data.columns=num_cols.columns
num_data.head()

Name,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,ct_flw_http_mthd,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,1.200687e-07,0.000010,0.000011,0.121569,0.114173,0.0,0.0,0.000095,0.004829,0.000196,0.000182,0.0,0.0,0.0,0.0,0.043883,0.054667,0.0,0.0,0.000000,0.000000,0.0013,0.0,2.832865e-07,2.185413e-07,0.0,0.0,0.0,0.0,0.0,0.046512,0.146341,0.00000,0.040816,0.000000,0.0,0.000000,0.0
1,4.112267e-06,0.000039,0.000021,0.121569,0.114173,0.0,0.0,0.000017,0.000392,0.000392,0.000365,0.0,0.0,0.0,0.0,0.087766,0.050667,0.0,0.0,0.000009,0.000014,0.0013,0.0,1.167307e-04,1.271630e-04,0.0,0.0,0.0,0.0,0.0,0.023256,0.073171,0.02439,0.040816,0.000000,0.0,0.027027,0.0
2,1.273525e-07,0.000011,0.000012,0.121569,0.114173,0.0,0.0,0.000099,0.004942,0.000196,0.000182,0.0,0.0,0.0,0.0,0.048537,0.059333,0.0,0.0,0.000000,0.000000,0.0013,0.0,2.832865e-07,2.185413e-07,0.0,0.0,0.0,0.0,0.0,0.255814,0.170732,0.00000,0.020408,0.028571,0.0,0.000000,0.0
3,1.375953e-07,0.000010,0.000011,0.121569,0.114173,0.0,0.0,0.000083,0.004214,0.000196,0.000182,0.0,0.0,0.0,0.0,0.043883,0.054667,0.0,0.0,0.000000,0.000000,0.0013,0.0,7.165482e-07,2.353522e-07,0.0,0.0,0.0,0.0,0.0,0.116279,0.195122,0.00000,0.000000,0.000000,0.0,0.000000,0.0
4,1.330429e-07,0.000011,0.000012,0.121569,0.114173,0.0,0.0,0.000095,0.004730,0.000196,0.000182,0.0,0.0,0.0,0.0,0.048537,0.059333,0.0,0.0,0.000000,0.000000,0.0013,0.0,8.331956e-08,5.043261e-08,0.0,0.0,0.0,0.0,0.0,0.139535,0.195122,0.00000,0.000000,0.000000,0.0,0.000000,0.0


In [28]:
cat_cols = data.select_dtypes(exclude='number')

In [31]:
cat_data=cat_cols.drop(['srcip','dstip','sport','dsport', 'attack_cat'], axis=1)
cat_data

Name,proto,state,service,is_sm_ips_ports,ct_state_ttl,is_ftp_login
0,udp,CON,dns,False,0,False
1,udp,CON,None,False,0,False
2,udp,CON,dns,False,0,False
3,udp,CON,dns,False,0,False
4,udp,CON,dns,False,0,False
...,...,...,...,...,...,...
699996,tcp,FIN,None,False,0,False
699997,tcp,FIN,http,False,0,False
699998,tcp,FIN,None,False,0,False
699999,udp,CON,None,False,0,False


In [40]:
cat_data['is_ftp_login']=cat_data['is_ftp_login'].astype('category')
cat_data['is_sm_ips_ports']=cat_data['is_sm_ips_ports'].astype('category')

In [41]:
data_dummies = pd.get_dummies(cat_data, drop_first=True)

In [42]:
data_dummies

,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_esp,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,...,proto_sat-mon,proto_sccopmce,proto_scps,proto_sctp,proto_sdrp,proto_secure-vmtp,proto_sep,proto_skip,proto_sm,proto_smp,proto_snp,proto_sprite-rpc,proto_sps,proto_srp,proto_st2,proto_stp,proto_sun-nd,proto_swipe,proto_tcf,proto_tcp,proto_tlsp,proto_tp++,proto_trunk-1,proto_trunk-2,proto_ttp,proto_udp,proto_udt,proto_unas,proto_uti,proto_vines,proto_visa,proto_vmtp,proto_vrrp,proto_wb-expak,proto_wb-mon,proto_wsn,proto_xnet,proto_xns-idp,proto_xtp,proto_zero,state_CLO,state_CON,state_ECO,state_ECR,state_FIN,state_INT,state_MAS,state_None,state_PAR,state_REQ,state_RST,state_TST,state_TXD,state_URH,state_URN,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,is_sm_ips_ports_True,ct_state_ttl_1,ct_state_ttl_2,ct_state_ttl_3,ct_state_ttl_4,ct_state_ttl_5,ct_state_ttl_6,is_ftp_login_True
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [44]:
df = pd.concat([data_dummies, num_data ], axis =1)
df

,proto_a/n,proto_aes-sp3-d,proto_any,proto_argus,proto_aris,proto_arp,proto_ax.25,proto_bbn-rcc,proto_bna,proto_br-sat-mon,proto_cbt,proto_cftp,proto_chaos,proto_compaq-peer,proto_cphb,proto_cpnx,proto_crtp,proto_crudp,proto_dcn,proto_ddp,proto_ddx,proto_dgp,proto_egp,proto_eigrp,proto_emcon,proto_encap,proto_esp,proto_etherip,proto_fc,proto_fire,proto_ggp,proto_gmtp,proto_gre,proto_hmp,proto_i-nlsp,proto_iatp,proto_ib,proto_icmp,proto_idpr,proto_idpr-cmtp,proto_idrp,proto_ifmp,proto_igmp,proto_igp,proto_il,proto_ip,proto_ipcomp,proto_ipcv,proto_ipip,proto_iplt,proto_ipnip,proto_ippc,proto_ipv6,proto_ipv6-frag,proto_ipv6-no,proto_ipv6-opts,proto_ipv6-route,proto_ipx-n-ip,proto_irtp,proto_isis,proto_iso-ip,proto_iso-tp4,proto_kryptolan,proto_l2tp,proto_larp,proto_leaf-1,proto_leaf-2,proto_merit-inp,proto_mfe-nsp,proto_mhrp,proto_micp,proto_mobile,proto_mtp,proto_mux,proto_narp,...,proto_xtp,proto_zero,state_CLO,state_CON,state_ECO,state_ECR,state_FIN,state_INT,state_MAS,state_None,state_PAR,state_REQ,state_RST,state_TST,state_TXD,state_URH,state_URN,service_dhcp,service_dns,service_ftp,service_ftp-data,service_http,service_irc,service_pop3,service_radius,service_smtp,service_snmp,service_ssh,service_ssl,is_sm_ips_ports_True,ct_state_ttl_1,ct_state_ttl_2,ct_state_ttl_3,ct_state_ttl_4,ct_state_ttl_5,ct_state_ttl_6,is_ftp_login_True,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,ct_flw_http_mthd,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.200687e-07,0.000010,0.000011,0.121569,0.114173,0.000000,0.000000,0.000095,0.004829,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.043883,0.054667,0.000,0.000000,0.000000,0.000000e+00,0.001300,0.0,2.832865e-07,2.185413e-07,0.000000,0.000000,0.000000,0.000000,0.0,0.046512,0.146341,0.000000,0.040816,0.000000,0.000000,0.000000,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.112267e-06,0.000039,0.000021,0.121569,0.114173,0.000000,0.000000,0.000017,0.000392,0.000392,0.000365,0.0,0.0,0.000000,0.000000,0.087766,0.050667,0.000,0.000000,0.000009,1.367440e-05,0.001300,0.0,1.167307e-04,1.271630e-04,0.000000,0.000000,0.000000,0.000000,0.0,0.023256,0.073171,0.024390,0.040816,0.000000,0.000000,0.027027,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.273525e-07,0.000011,0.000012,0.121569,0.114173,0.000000,0.000000,0.000099,0.004942,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.048537,0.059333,0.000,0.000000,0.000000,0.000000e+00,0.001300,0.0,2.832865e-07,2.185413e-07,0.000000,0.000000,0.000000,0.000000,0.0,0.255814,0.170732,0.000000,0.020408,0.028571,0.000000,0.000000,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.375953e-07,0.000010,0.000011,0.121569,0.114173,0.000000,0.000000,0.000083,0.004214,0.000196,0.000182,0.0,0.0,0.000000,0.000000,0.043883,0.054667,0.000,0.000000,0.000000,0.000000e+00,0.001300,0.0,7.165482e-07,2.353522e-07,0.000000,0.000000,0.000000,0.000000,0.0,0.116279,0.195122,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [48]:
labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anomaly_labels.csv')
labels.Label = [-x if x == 1 else 1 for x in labels.Label ]
anomaly_labels = labels.Label

### Full Model

In [49]:
iso_model_full = IsolationForest(random_state=0).fit(df)

anomaly_preds_full = iso_model_full.predict(df)
pd.crosstab(anomaly_labels, anomaly_preds_full)

col_0,-1,1
Label,,
-1,20602,1613
1,28815,648971


In [50]:
calinski_harabasz_score(df, anomaly_preds_full)

15918.944882320375

In [51]:
davies_bouldin_score(df, anomaly_preds_full)

3.7649187725069173

### Model with Selected Features

In [54]:
service_dummy=pd.get_dummies(data['service'], drop_first=True)
selected_df=pd.concat([num_data[['sttl', 'sbytes', 'smeansz', 'ct_dst_sport_ltm']], service_dummy], axis=1)

In [55]:
selected_df

,sttl,sbytes,smeansz,ct_dst_sport_ltm,dhcp,dns,ftp,ftp-data,http,irc,pop3,radius,smtp,snmp,ssh,ssl
0,0.121569,0.000010,0.043883,0.000000,0,1,0,0,0,0,0,0,0,0,0,0
1,0.121569,0.000039,0.087766,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
2,0.121569,0.000011,0.048537,0.000000,0,1,0,0,0,0,0,0,0,0,0,0
3,0.121569,0.000010,0.043883,0.000000,0,1,0,0,0,0,0,0,0,0,0,0
4,0.121569,0.000011,0.048537,0.000000,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699996,0.121569,0.000023,0.035239,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
699997,0.121569,0.001419,0.035239,0.030303,0,0,0,0,1,0,0,0,0,0,0,0
699998,0.121569,0.000158,0.059840,0.000000,0,0,0,0,0,0,0,0,0,0,0,0
699999,0.121569,0.000042,0.094415,0.000000,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
iso_model_selected = IsolationForest(random_state=0).fit(selected_df)

anomaly_preds_selected = iso_model_selected.predict(selected_df)
pd.crosstab(anomaly_labels, anomaly_preds_selected)

col_0,-1,1
Label,,
-1,14896,7319
1,81084,596702


In [57]:
calinski_harabasz_score(selected_df, anomaly_preds_selected)

34444.326994613715

In [58]:
davies_bouldin_score(selected_df, anomaly_preds_selected)

3.399364374383159

### Model 1

In [ ]:
df1=df[['sbytes', 'sttl']].copy()

In [46]:
iso_model1 = IsolationForest(random_state=0).fit(df1)

anomaly_preds1 = iso_model1.predict(df1)
pd.crosstab(anomaly_labels, anomaly_preds1)

col_0,-1,1
Label,,
-1,22215,0
1,84639,593147


In [47]:
calinski_harabasz_score(df1, anomaly_preds1)

170577.33528979166

In [48]:
davies_bouldin_score(df1, anomaly_preds1)

1.52104188768819

### Model 2

In [50]:
df2=df[['sbytes', 'sttl', 'dttl']].copy()

In [51]:
iso_model2 = IsolationForest(random_state=0).fit(df2)

anomaly_preds2 = iso_model2.predict(df2)
pd.crosstab(anomaly_labels, anomaly_preds2)

col_0,-1,1
Label,,
-1,22215,0
1,83556,594230


In [52]:
calinski_harabasz_score(df2, anomaly_preds2)

118616.59128718833

In [53]:
davies_bouldin_score(df2, anomaly_preds2)

1.8060416179297087

### Model3

In [54]:
df3=df[['sbytes', 'sttl','tcprtt','synack']].copy()
iso_model3 = IsolationForest(random_state=0).fit(df3)

anomaly_preds3 = iso_model3.predict(df3)
pd.crosstab(anomaly_labels, anomaly_preds3)

col_0,-1,1
Label,,
-1,22173,42
1,43721,634065


In [55]:
calinski_harabasz_score(df3, anomaly_preds3)

388570.03074713424

In [56]:
davies_bouldin_score(df3, anomaly_preds3)

1.2116985450652717

### Model 4

In [60]:
df4=df[['sbytes', 'sttl','tcprtt','synack', 'ackdat']].copy()
iso_model4 = IsolationForest(random_state=0).fit(df4)

anomaly_preds4 = iso_model4.predict(df4)
pd.crosstab(anomaly_labels, anomaly_preds4)

col_0,-1,1
Label,,
-1,22099,116
1,25458,652328


In [61]:
calinski_harabasz_score(df4, anomaly_preds4)

655861.022311861

In [62]:
davies_bouldin_score(df4, anomaly_preds4)

0.9897927556404994

### Model 5

In [60]:
df5=df[['sbytes', 'sttl','tcprtt','synack', 'ackdat', 'Sload']].copy()
iso_model5 = IsolationForest(random_state=0).fit(df5)

anomaly_preds5 = iso_model5.predict(df5)
pd.crosstab(anomaly_labels, anomaly_preds5)

col_0,-1,1
Label,,
-1,22096,119
1,34157,643629


In [61]:
calinski_harabasz_score(df5, anomaly_preds5)

528545.852362087

In [62]:
davies_bouldin_score(df5, anomaly_preds5)

1.0802564711317224

#### We have obtained best scores in Model 4 with `sbytes`, `sttl`,`tcprtt`,`synack`, `ackdat` attributes. Therefore we use these attributes for hyperparameter tuning. 

### Hyperparameter Tuning

In [63]:
params = {'n_estimators': [100, 200, 300, 400],
         'max_samples': [100, 200, 300, 400]}

score_table = pd.DataFrame(columns=['Number of Estimators', 'Max Sample Size',
                                    'Calinski Harabasz Score', 'Davies Bouldin Score'])


for num_est in params['n_estimators']:
    for maxsamp in params['max_samples']:
        iso_model = IsolationForest(random_state=0, n_estimators=num_est, max_samples=maxsamp).fit(df4)
        anomaly_preds = iso_model.predict(df4)
        calinski = calinski_harabasz_score(df4, anomaly_preds)
        davies = davies_bouldin_score(df4, anomaly_preds)
        score_table=score_table.append({'Number of Estimators':num_est, 
                                        'Max Sample Size':maxsamp,
                                        'Calinski Harabasz Score':calinski, 
                                        'Davies Bouldin Score':davies}, ignore_index=True)
        
display(score_table)

,Number of Estimators,Max Sample Size,Calinski Harabasz Score,Davies Bouldin Score
0,100.0,100.0,212698.003289,1.497802
1,100.0,200.0,565381.621139,1.059028
2,100.0,300.0,571976.415150,1.056813
3,100.0,400.0,984355.084638,0.801441
4,200.0,100.0,240602.075855,1.426563
5,200.0,200.0,477550.376259,1.143036
6,200.0,300.0,538642.895881,1.085799
7,200.0,400.0,707633.079614,0.953299
8,300.0,100.0,247072.151662,1.410777
9,300.0,200.0,458090.835023,1.161898


## PCA

In [2]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

# We get the components by 
# calling fit_transform method with our data
pca_components = pca.fit_transform(df)

NameError: name 'df' is not defined

In [ ]:
pca_components.shape

In [ ]:
sns.scatterplot(x=pca_components[:, 0],y= pca_components[:, 1], hue=anomaly_labels)

In [ ]:
sns.scatterplot(x=pca_components[:, 0],y= pca_components[:, 1], hue=anomaly_preds4)

In [ ]:
sns.scatterplot(df.sbytes, df.sttl, hue=anomaly_preds4)